In [1]:
import pandas as pd
import numpy as np


![任务1](https://upload-images.jianshu.io/upload_images/25239821-2ff2657f1b5eba51.png)

In [2]:
df = pd.read_table('./final_test_data/benchmark.txt', header = None)
df.head(5)

,0
0,start
1,benchmark start : 2020/12/24 12:12:48
2,Number of GPUs on current device : 1
3,CUDA Version : 11.0
4,Cudnn Version : 8005


In [25]:
pat1='Benchmarking (?P<type_x>\w+) (?P<precision>\w+) precision type (?P<model_name_x>\w+)'
pat2='(?P<model_name_y>\w+)  model average (?P<type_y>\w+) time :  (?P<time_val>.+) ms'
bench_info=df[0].str.extract(pat1).dropna().reset_index(drop=True)
bench_info

,type_x,precision,model_name_x
0,Training,float,mnasnet0_5
1,Training,float,mnasnet0_75
2,Training,float,mnasnet1_0
3,Training,float,mnasnet1_3
4,Training,float,resnet18
...,...,...,...
187,Inference,double,mobilenet_v2
188,Inference,double,shufflenet_v2_x0_5
189,Inference,double,shufflenet_v2_x1_0
190,Inference,double,shufflenet_v2_x1_5


In [16]:
model_info=df[0].str.extract(pat2).dropna().reset_index(drop=True)
model_info

,model_name_y,type_y,time_val
0,mnasnet0_5,train,28.527636528015137
1,mnasnet0_75,train,34.10548686981201
2,mnasnet1_0,train,34.31377410888672
3,mnasnet1_3,train,35.556888580322266
4,resnet18,train,18.660082817077637
...,...,...,...
187,mobilenet_v2,inference,25.47874927520752
188,shufflenet_v2_x0_5,inference,11.498098373413086
189,shufflenet_v2_x1_0,inference,12.888350486755371
190,shufflenet_v2_x1_5,inference,21.520133018493652


In [17]:
tmp = pd.concat([bench_info, model_info], axis = 1).reset_index(drop = True)
tmp

,type_x,precision,model_name_x,model_name_y,type_y,time_val
0,Training,float,mnasnet0_5,mnasnet0_5,train,28.527636528015137
1,Training,float,mnasnet0_75,mnasnet0_75,train,34.10548686981201
2,Training,float,mnasnet1_0,mnasnet1_0,train,34.31377410888672
3,Training,float,mnasnet1_3,mnasnet1_3,train,35.556888580322266
4,Training,float,resnet18,resnet18,train,18.660082817077637
...,...,...,...,...,...,...
187,Inference,double,mobilenet_v2,mobilenet_v2,inference,25.47874927520752
188,Inference,double,shufflenet_v2_x0_5,shufflenet_v2_x0_5,inference,11.498098373413086
189,Inference,double,shufflenet_v2_x1_0,shufflenet_v2_x1_0,inference,12.888350486755371
190,Inference,double,shufflenet_v2_x1_5,shufflenet_v2_x1_5,inference,21.520133018493652


In [26]:
tmp=tmp[['model_name_x','type_y','precision',
        'time_val']].rename(columns={'model_name_x':'model','type_y':'type'})
tmp.time_val=pd.to_numeric(tmp.time_val, errors='coerce').round(3)
tmp


,model,type,precision,time_val
0,mnasnet0_5,train,float,28.528
1,mnasnet0_75,train,float,34.105
2,mnasnet1_0,train,float,34.314
3,mnasnet1_3,train,float,35.557
4,resnet18,train,float,18.660
...,...,...,...,...
187,mobilenet_v2,inference,double,25.479
188,shufflenet_v2_x0_5,inference,double,11.498
189,shufflenet_v2_x1_0,inference,double,12.888
190,shufflenet_v2_x1_5,inference,double,21.520


In [28]:
#变型 长变宽
tmp=tmp.pivot(index=['model','type'], columns='precision', values='time_val')
tmp


precision                     double    float     half
model            type                                 
densenet121      inference   144.111   15.637   19.772
                 train       417.207   93.357   88.976
densenet161      inference   511.177   31.750   27.555
                 train      1290.287  136.624  144.319
densenet169      inference   175.808   21.598   26.371
...                              ...      ...      ...
vgg19_bn         train      2463.644   64.083   55.610
wide_resnet101_2 inference   982.664   36.296   20.252
                 train      2940.320  111.306  101.254
wide_resnet50_2  inference   495.217   21.209   10.985
                 train      1509.889   64.756   58.925

[64 rows x 3 columns]

In [30]:
tmp=tmp.unstack()
tmp.head()



precision      double               float               half         
type        inference     train inference    train inference    train
model                                                                
densenet121   144.111   417.207    15.637   93.357    19.772   88.976
densenet161   511.177  1290.287    31.750  136.624    27.555  144.319
densenet169   175.808   511.404    21.598  104.840    26.371  121.556
densenet201   223.960   654.365    26.169  129.334    33.394  118.940
mnasnet0_5     11.870    48.232     8.039   28.528     6.929   27.198

In [32]:
tmp.columns=tmp.columns.map(lambda x:(x[1]+'_'+x[0]))
tmp=tmp.reset_index()
tmp.head()


,model,inference_double,train_double,inference_float,train_float,inference_half,train_half
0,densenet121,144.111,417.207,15.637,93.357,19.772,88.976
1,densenet161,511.177,1290.287,31.750,136.624,27.555,144.319
2,densenet169,175.808,511.404,21.598,104.840,26.371,121.556
3,densenet201,223.960,654.365,26.169,129.334,33.394,118.940
4,mnasnet0_5,11.870,48.232,8.039,28.528,6.929,27.198


In [33]:
# temp.sort_index(axis=1,ascending=False)
tmp=tmp[['model','train_half','train_float',
           'train_double','inference_half',
           'inference_float','inference_double']]
tmp=tmp.sort_values('model',ascending=True)
tmp.head()


,model,train_half,train_float,train_double,inference_half,inference_float,inference_double
0,densenet121,88.976,93.357,417.207,19.772,15.637,144.111
1,densenet161,144.319,136.624,1290.287,27.555,31.750,511.177
2,densenet169,121.556,104.840,511.404,26.371,21.598,175.808
3,densenet201,118.940,129.334,654.365,33.394,26.169,223.960
4,mnasnet0_5,27.198,28.528,48.232,6.929,8.039,11.870


![任务2](https://upload-images.jianshu.io/upload_images/25239821-81b3f0335c2628cc.png)

In [41]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('./final_test_data/yali18.csv')
df2 = pd.read_csv('./final_test_data/yali19.csv')
df3 = pd.read_csv('./final_test_data/qx1819.csv')
print(df1.head(5))
print(df2.head(5))
print(df3.head(5))

         Time MeasName        H0        H1        H2        H3        H4  \
0  2018-01-01      站点4  0.402750  0.407625  0.418125  0.425250  0.426000   
1  2018-01-01      站点7  0.214375  0.226750  0.232375  0.233125  0.235000   
2  2018-01-01     站点22  0.247000  0.248125  0.271375  0.251125  0.272125   
3  2018-01-01     站点21  0.284250  0.289875  0.283500  0.281250  0.288375   
4  2018-01-01     站点20  0.292875  0.295875  0.305250  0.298875  0.310125   

         H5        H6        H7  ...       H14       H15       H16       H17  \
0  0.425250  0.417375  0.426375  ...  0.348750  0.359250  0.355500  0.344250   
1  0.232750  0.230875  0.220000  ...  0.187375  0.196750  0.199750  0.192250   
2  0.256375  0.257125  0.242500  ...  0.245500  0.242875  0.238375  0.230875   
3  0.288750  0.285750  0.255750  ...  0.227625  0.238125  0.238500  0.218625   
4  0.300750  0.288375  0.262500  ...  0.247500  0.241125  0.243375  0.232500   

        H18       H19       H20       H21       H22       H23 

In [42]:
#宽表变长表
# H_list=['H'+str(i) for i in range(12)]
# df1=df1.melt(id_vars =['Time','MeasName'],
#              value_vars =H_list,var_name = 'H',
#              value_name = '压力')
# df1
#宽表变长表
H_list=['H'+str(i) for i in range(24)]
df1=df1.melt(id_vars =['Time','MeasName'],
             value_vars =H_list,var_name = 'H',
             value_name = '压力')
df1


,Time,MeasName,H,压力
0,2018-01-01,站点4,H0,0.402750
1,2018-01-01,站点7,H0,0.214375
2,2018-01-01,站点22,H0,0.247000
3,2018-01-01,站点21,H0,0.284250
4,2018-01-01,站点20,H0,0.292875
...,...,...,...,...
262795,2018-12-31,站点12,H23,0.179500
262796,2018-12-31,站点17,H23,0.185625
262797,2018-12-31,站点2,H23,0.234125
262798,2018-12-31,站点9,H23,0.246375


In [43]:
# 修改对应格式 str正则
df1.MeasName=df1.MeasName.str.replace('站点','')
df1.H=df1.H.str.replace('H','')
df1.Time=df1.Time+'-'+df1.H
df1.Time=pd.to_datetime(df1.Time,format='%Y-%m-%d-%H')
df1=df1.rename(columns={'MeasName':'站点'}).drop(columns='H')
df1


,Time,站点,压力
0,2018-01-01 00:00:00,4,0.402750
1,2018-01-01 00:00:00,7,0.214375
2,2018-01-01 00:00:00,22,0.247000
3,2018-01-01 00:00:00,21,0.284250
4,2018-01-01 00:00:00,20,0.292875
...,...,...,...
262795,2018-12-31 23:00:00,12,0.179500
262796,2018-12-31 23:00:00,17,0.185625
262797,2018-12-31 23:00:00,2,0.234125
262798,2018-12-31 23:00:00,9,0.246375
